In [ ]:
# 코랩 환경에서 'evaluate' 라이브러리가 설치되지 않았다면, 아래 셀의 주석을 해제하고 실행하세요.
# 설치 후에는 런타임을 다시 시작할 필요 없이 바로 아래 코드를 실행할 수 있습니다.
# !pip install evaluate

# 필요한 라이브러리를 임포트합니다.
import numpy as np # 숫자 계산을 위한 라이브러리
import torch # PyTorch 딥러닝 프레임워크 (버전 1.8.2 이상 필요)

# Hugging Face의 transformers 라이브러리에서 AutoTokenizer와 AutoModelForSequenceClassification을 임포트합니다.
# AutoTokenizer: 사전 학습된 토크나이저를 모델 이름으로 쉽게 로드합니다.
# AutoModelForSequenceClassification: 시퀀스 분류를 위한 사전 학습된 모델을 로드합니다.
from transformers import AutoTokenizer, AutoModelForSequenceClassification ## 버전 4.12.3 필요

# Hugging Face의 datasets 라이브러리를 임포트합니다. 데이터셋 로딩 및 처리에 사용됩니다.
import datasets ## 버전 2.1.0 필요

# Hugging Face의 evaluate 라이브러리를 임포트합니다. 평가 지표 로드에 사용됩니다. (datasets 2.3.0 버전 이후 분리됨)
import evaluate # evaluate 라이브러리를 임포트합니다.

# --- 데이터 로딩 및 전처리 ---

# datasets 라이브러리에서 load_dataset 함수를 임포트합니다.
from datasets import load_dataset

# "searle-j/kote" 데이터셋을 로드합니다.
# 기본 설정인 'dichotomized'가 사용됩니다.
print("데이터셋 로드 시작...")
dataset = load_dataset("searle-j/kote")
print("데이터셋 로드 완료.")

# 로드된 데이터셋의 구조를 확인합니다 (train, test, validation 스플릿 포함).
# print(dataset) # 필요하다면 이 줄의 주석을 해제하여 데이터셋 구조 확인

# 학습 데이터셋의 샘플 하나를 확인합니다.
# print(dataset["train"][25597]) # 필요하다면 이 줄의 주석을 해제하여 샘플 확인

# 다중 레이블(multi-hot labels, 44차원)을 생성합니다.

# scikit-learn의 preprocessing 모듈에서 MultiLabelBinarizer를 임포트합니다.
# 리스트 형태의 레이블을 이진 벡터(멀티-핫 인코딩)로 변환하는 데 사용됩니다.
from sklearn.preprocessing import MultiLabelBinarizer

print("레이블 이진화 시작...")
# MultiLabelBinarizer 객체를 생성합니다.
mlb = MultiLabelBinarizer()

# 학습, 테스트, 검증 데이터셋의 'labels' 컬럼을 MultiLabelBinarizer를 사용하여 변환합니다.
# fit_transform은 레이블 클래스를 학습하고 데이터를 변환합니다.
train_labels = mlb.fit_transform(dataset["train"]["labels"])
test_labels = mlb.fit_transform(dataset["test"]["labels"])
val_labels = mlb.fit_transform(dataset["validation"]["labels"])
print("레이블 이진화 완료.")

# 변환된 레이블 배열의 형태(shape)를 출력하여 확인합니다.
print("train_labels shape ::: {}".format(train_labels.shape))
print("test_labels shape :::: {}".format(test_labels.shape))
print("val_labels shape ::::: {}".format(val_labels.shape))
print("\ngood!") # 형태가 올바른지 확인 메시지 출력

# 변환된 멀티-핫 레이블을 데이터셋에 새로운 컬럼으로 추가합니다.

print("바이너리 레이블 컬럼 추가 시작...")
# 학습, 테스트, 검증 스플릿에 'binary_labels' 컬럼을 추가합니다.
# NumPy 배열을 Python 리스트로 변환하여 추가합니다 (.tolist()).
dataset["train"] = dataset["train"].add_column("binary_labels", train_labels.tolist())
dataset["test"] = dataset["test"].add_column("binary_labels", test_labels.tolist())
dataset["validation"] = dataset["validation"].add_column("binary_labels", val_labels.tolist())
print("바이너리 레이블 컬럼 추가 완료.")

# 학습 데이터셋의 피처(컬럼) 키를 확인합니다. 'binary_labels'가 추가되었는지 봅니다.
# print(dataset["train"].features.keys()) # 필요하다면 이 줄의 주석을 해제하여 피처 확인


# split indicator columns 추가 <-- 이 부분이 map 함수 호출 전에 실행되어야 합니다.
print("split 컬럼 추가 시작...")
dataset["train"] = dataset["train"].add_column("split", ["train"]*len(dataset["train"]))
dataset["test"] = dataset["test"].add_column("split", ["test"]*len(dataset["test"]))
dataset["validation"] = dataset["validation"].add_column("split", ["validation"]*len(dataset["validation"]))
print("split 컬럼 추가 완료.")


# --- 토큰화 및 데이터 증강 ---

# Hugging Face에서 사전 학습된 토크나이저를 다운로드하여 로드합니다.

# 사용할 모델의 이름입니다. 한국어 모델인 'beomi/KcELECTRA-base'를 사용합니다.
MODEL_NAME = 'beomi/KcELECTRA-base' # <-- 감사합니다! (원문 주석)
print(f"'{MODEL_NAME}' 토크나이저 로드 시작...")
# 모델 이름으로 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("토크나이저 로드 완료.")


# ### 샘플 인코딩 (Sample Encoding) - 데모 목적으로 사용되었으며, 실제 파이프라인 실행 시에는 필요하지 않습니다.
# 아래 블록은 sample_text 변수를 사용하므로 주석 처리합니다.

# # 샘플 텍스트가 PyTorch 텐서로 어떻게 변환되는지 확인합니다.
# # 학습 데이터셋의 샘플 텍스트 하나를 가져옵니다.
# sample_text = dataset["train"]["text"][25597] # 이 줄은 이제 사용되지 않습니다.
# sample_text

# # 샘플 텍스트를 토크나이저로 인코딩합니다.
# encoding = tokenizer.encode_plus(
#     sample_text, # 인코딩할 텍스트
#     add_special_tokens=True,
#     max_length=512,
#     return_token_type_ids=False,
#     padding="max_length",
#     return_attention_mask=True,
#     return_tensors='pt',
# )

# # 인코딩 결과의 키를 확인합니다.
# # encoding.keys()

# # 인코딩 결과 상세 보기
# # encoding

# # 인코딩된 텐서의 형태를 확인합니다.
# # encoding["input_ids"].shape, encoding["attention_mask"].shape

# # 토큰 ID를 다시 토큰 문자열로 변환하여 확인할 수 있습니다.
# print(tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze()))

# ### 학습 인스턴스를 위한 토큰 치환 및 마스킹 (Token Switching and Token Masking for the Training Instances)

# 더 나은 성능을 위해 학습 데이터셋의 일부 토큰을 치환하거나 마스킹합니다. (데이터 증강)


# 토큰 마스킹 함수 정의
# encoding: 토크나이저 인코딩 결과 (여기서는 _preprocess 함수 내에서 사용될 encoding)
# prob: 마스킹할 확률
def token_masking(encoding, prob):
    # 인코딩된 input_ids 텐서의 각 토큰을 순회합니다.
    # 주의: 이 함수는 배치 처리 함수 내에서 호출될 때 올바르게 작동하도록 수정이 필요할 수 있습니다.
    # 원본 Notebook 코드를 그대로 유지하며 주석을 추가합니다. 배치 처리를 고려한 수정은 별도로 필요할 수 있습니다.
    for i, token_tensor in enumerate(encoding['input_ids']): # 각 배치 항목(시퀀스) 순회
        for j, token in enumerate(token_tensor): # 시퀀스 내의 각 토큰 순회
            # 특수 토큰([PAD]=0, [UNK]=1, [CLS]=2, [SEP]=3)이 아닌 경우에만 마스킹 대상이 됩니다.
            if token not in [0, 1, 2, 3]:
                # 0에서 1 사이의 균등 분포 난수가 prob보다 작으면 마스킹을 수행합니다.
                if np.random.uniform(0, 1) < prob:
                    # 해당 토큰 ID를 [MASK] 토큰 ID (4)로 변경합니다.
                    encoding['input_ids'][i][j] = 4 # 4는 '[MASK]'입니다. # 올바른 배치/토큰 인덱싱
    return encoding # 변경된 인코딩 결과를 반환합니다.

# 토큰 치환 함수 정의
# encoding: 토크나이저 인코딩 결과 (여기서는 _preprocess 함수 내에서 사용될 encoding)
# prob: 치환할 확률
def token_switching(encoding, prob):
    # 인코딩된 input_ids 텐서의 각 토큰을 순회합니다.
    # token_masking 함수와 마찬가지로 배치 처리를 고려한 인덱싱 수정이 필요할 수 있습니다.
    for i, token_tensor in enumerate(encoding['input_ids']): # 각 배치 항목(시퀀스) 순회
         for j, token in enumerate(token_tensor): # 시퀀스 내의 각 토큰 순회
            # 특수 토큰 및 [MASK] 토큰([PAD]=0, [UNK]=1, [CLS]=2, [SEP]=3, [MASK]=4)이 아닌 경우에만 치환 대상이 됩니다.
            if token not in [0, 1, 2, 3, 4]:
                # 0에서 1 사이의 균등 분포 난수가 prob보다 작으면 치환을 수행합니다.
                if np.random.uniform(0, 1) < prob:
                    # 어휘집 크기 범위 내에서 특수 토큰을 제외한 임의의 다른 토큰 ID를 선택하여 해당 토큰 ID를 변경합니다.
                    encoding['input_ids'][i][j] = np.random.choice(np.arange(5, tokenizer.vocab_size), 1)[0] # 올바른 배치/토큰 인덱싱
    return encoding # 변경된 인코딩 결과를 반환합니다.


# 마스킹과 치환을 함께 적용하는 함수 정의
# encoding: 토크나이저 인코딩 결과
# prob: 마스킹 또는 치환될 총 확률 (각각 prob/2 확률로 적용됨)
def mask_and_switch(encoding, prob: float = 0.1):
    # 총 확률의 절반으로 토큰 마스킹을 적용합니다.
    encoding = token_masking(encoding, prob / 2)
    # 총 확률의 절반으로 토큰 치환을 적용합니다.
    encoding = token_switching(encoding, prob / 2)

    return encoding # 마스킹 및 치환이 적용된 인코딩 결과를 반환합니다.

# 마스킹 및 치환 데모 (sample_text 데모와 함께 주석 처리)
# # see some tokens in the sample text has changed...
# encoding = tokenizer.encode_plus(
#     sample_text,
#     add_special_tokens=True,
#     max_length=512,
#     return_token_type_ids=False,
#     padding="max_length",
#     return_attention_mask=True,
#     return_tensors='pt',
# ) # 데모를 위해 다시 인코딩
# encoding = mask_and_switch(encoding, prob=0.1)
# print(tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())) # 일부 토큰이 변경된 것을 볼 수 있습니다.


# ### 실제 데이터에 대한 실제 인코딩 (Real Encoding for Real Data)

# 테스트 및 검증 데이터셋의 토큰은 변경하지 않기 위해 split 구분자를 추가합니다.
# 이 부분은 이미 위에서 데이터셋 로드 후 진행되었습니다.


# 데이터 전처리 함수 정의
# instance: 데이터셋의 한 샘플 (딕셔너리 형태), datasets 라이브러리의 map 함수가 배치 단위로 호출 시 각 키의 값이 리스트 또는 텐서 형태가 됩니다.
# prob: 마스킹 및 치환 확률 (학습 데이터에만 적용)
# max_length: 최대 시퀀스 길이
def _preprocess(instance, prob: float = 0.1, max_length: int = 512):
    # 텍스트를 토큰화합니다.
    # batched=True 시 instance["text"]는 텍스트들의 리스트가 됩니다.
    encoded = tokenizer(
        instance["text"], # 토큰화할 텍스트 (배치 시 리스트)
        max_length=max_length, # 최대 길이 설정
        padding="max_length", # 최대 길이에 맞춰 패딩
        truncation=True, # 최대 길이 초과 시 자르기
        return_token_type_ids=True # token_type_ids를 반환하도록 설정 (ELECTRA 모델에 필요할 수 있음)
    )
    # 토큰 마스킹 및 토큰 치환을 적용합니다.
    # 현재 인스턴스가 학습 데이터인 경우에만 mask_and_switch 함수를 적용합니다.
    # batched=True로 호출되므로 instance["split"]는 split 값들의 리스트가 됩니다.
    # 배치 내 모든 샘플이 'train' 스플릿일 때만 증강을 적용합니다.
    # 만약 배치가 여러 스플릿을 포함할 수 있다면 로직 수정이 필요하지만, 일반적으로 스플릿 별로 map을 호출하므로 괜찮습니다.
    if instance["split"][0] == "train": # 배치 내 첫 번째 샘플의 스플릿으로 판단
        # mask_and_switch 함수는 단일 시퀀스 처럼 구현되어 있으므로 배치 처리 시 주의 필요 (위 mask/switch 함수 주석 참고)
        # 여기서는 원본 Notebook 코드를 따르지만, map(batched=True)와 함께 사용할 경우 mask_and_switch 함수 내의 로직 수정이 권장됨
        encoded = mask_and_switch(encoded, prob=prob)
    else:
        pass # 테스트 또는 검증 데이터는 변경하지 않습니다.

    # 바이너리 레이블을 인코딩 결과에 추가합니다.
    # instance["binary_labels"]는 배치 시 바이너리 레이블 리스트의 리스트가 됩니다.
    # Features에서 float32로 명시했으므로 형변환 될 것입니다.
    encoded["binary_labels"] = instance["binary_labels"]

    return encoded # 전처리된 결과 (토큰 ID, 어텐션 마스크, 토큰 타입 ID, 바이너리 레이블)를 반환합니다.


# datasets 라이브러리에서 Features, Value, Sequence를 임포트합니다.
# 데이터셋의 컬럼별 데이터 타입을 명시적으로 정의하는 데 사용됩니다.
from datasets import Features, Value, Sequence

# 결과 데이터셋의 피처 구조를 정의합니다.
features = Features({
    "ID": Value("string"), # ID 컬럼 (문자열)
    "text": Value("string"), # 텍스트 컬럼 (문자열)
    "input_ids": Sequence(Value("int64")), # 입력 토큰 ID 시퀀스 (정수)
    "attention_mask": Sequence(Value("int64")), # 어텐션 마스크 시퀀스 (정수)
    "token_type_ids": Sequence(Value("int64")), # 토큰 타입 ID 시퀀스 (정수)
    "labels": Sequence(Value("int64")), # 원본 레이블 ID 시퀀스 (정수)
    "binary_labels": Sequence(Value("float32")), # 바이너리 레이블 시퀀스 (부동 소수점형으로 강제 변환)
    "split": Value("string") # 스플릿 구분자 컬럼 (문자열)
})

# dataset.map(_preprocess, ...) 호출 전에 데이터셋의 컬럼 확인
print("map 함수 호출 전 데이터셋 컬럼 확인:")
print("Train 스플릿 컬럼:", dataset["train"].column_names)
print("Validation 스플릿 컬럼:", dataset["validation"].column_names)
print("Test 스플릿 컬럼:", dataset["test"].column_names)


print("map 함수를 사용한 데이터셋 전처리 시작...")
# 데이터셋 전체에 _preprocess 함수를 매핑하여 적용합니다.
# features=features: 결과 데이터셋의 피처 구조를 위에서 정의한대로 설정합니다.
# batched=True: 배치 단위로 함수를 적용하여 처리 속도를 높입니다.
# batch_size=2: 한 번에 처리할 배치의 크기를 설정합니다. (메모리 사용량에 따라 조절 필요)
# 여기서의 batch_size는 dataset.map의 배치 크기이며, Trainer의 배치 크기와 다릅니다.
tokenized_dataset = dataset.map(_preprocess, features=features, batched=True, batch_size=2) # 이 라인에서 오류 발생 시 위의 print 문 확인
print("map 함수를 사용한 데이터셋 전처리 완료.")


# 토큰화된 데이터셋의 형식을 PyTorch 텐서로 설정하고 학습에 필요한 컬럼만 남깁니다.
print("데이터셋 형식 PyTorch 텐서로 변경 및 컬럼 정리 시작...")
# type="torch": PyTorch 텐서로 변환합니다.
# columns: PyTorch 텐서로 변환할 컬럼 목록입니다.
tokenized_dataset = tokenized_dataset.with_format(type="torch", columns=["input_ids","attention_mask","token_type_ids","binary_labels"])

# 학습에 불필요한 원본 컬럼을 제거합니다.
tokenized_dataset = tokenized_dataset.remove_columns(["ID","text","labels"])

# 'binary_labels' 컬럼의 이름을 'labels'로 변경하여 Trainer가 기본적으로 기대하는 이름과 일치시킵니다.
tokenized_dataset = tokenized_dataset.rename_column("binary_labels","labels")
print("데이터셋 형식 변경 및 컬럼 정리 완료.")


# 학습 데이터셋의 레이블 컬럼 데이터 타입을 확인합니다. float32인지 봅니다.
# print(tokenized_dataset["train"]["labels"].dtype) # 필요하다면 이 줄의 주석을 해제하여 타입 확인


# --- 모델 설정 ---

# 먼저, 원본 데이터셋에서 레이블 이름을 가져옵니다.
LABELS = dataset["train"].features["labels"].feature.names
# print(LABELS) # 레이블 이름 목록 출력 (필요하다면 주석 해제)

# 사전 학습된 모델을 로드합니다.

# 사용할 모델의 이름입니다. 한국어 모델인 'beomi/KcELECTRA-base'를 사용합니다.
MODEL_NAME = 'beomi/KcELECTRA-base'
print(f"'{MODEL_NAME}' 모델 로드 시작...")
# 시퀀스 분류를 위한 모델을 로드합니다.
# problem_type="multi_label_classification": 다중 레이블 분류 문제임을 명시합니다.
# num_labels=len(LABELS): 출력 레이어의 뉴런 수를 레이블 개수(44)와 일치시킵니다.
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    problem_type="multi_label_classification",
    num_labels=len(LABELS),
)
print("모델 로드 완료.")

# 모델의 설정을 확인합니다.
# print(model.config) # 필요하다면 주석 해제하여 설정 확인

## 모델 설정에 레이블 이름을 정의합니다.
# id를 레이블 이름으로 매핑하는 딕셔너리를 생성하여 모델 설정에 추가합니다.
model.config.id2label = {i:label for i,label in enumerate(LABELS)}
# 레이블 이름을 id로 매핑하는 딕셔너리를 생성하여 모델 설정에 추가합니다.
model.config.label2id = {label:i for i,label in enumerate(LABELS)}
print("모델 설정에 레이블 정보 추가 완료.")


# --- 학습 설정 ---

# Hugging Face의 TrainingArguments 클래스를 임포트합니다. 학습 관련 하이퍼파라미터 및 설정을 정의합니다.
from transformers import TrainingArguments

# GPU 당 배치 크기 (주석 처리됨, GPU가 여러 개 있는 경우 활성화 가능)
# BATCH_SIZE_per_device = 2  # <-- 부자라서 GPU가 여러 개 있다면 이 줄을 활성화하세요. (원문 주석)
# 총 학습 에포크 수
EPOCHS = 15

print("TrainingArguments 설정 시작...")
# TrainingArguments 객체를 생성하여 학습 설정을 정의합니다.
args = TrainingArguments(
    output_dir="kote_output", # 학습 중 생성되는 파일(체크포인트, 로그 등)을 저장할 디렉토리
    eval_strategy="epoch", # 최신 버전: 매 에포크 종료 시 평가 수행
#    per_device_train_batch_size=BATCH_SIZE_per_device, # GPU 당 학습 배치 크기 (설정 시)
#    per_device_eval_batch_size=BATCH_SIZE_per_device, # GPU 당 평가 배치 크기 (설정 시)
    num_train_epochs=EPOCHS, # 총 학습 에포크 수
    save_strategy="epoch", # 매 에포크 종료 시 모델 저장
    load_best_model_at_end=True, # 학습 종료 시 평가 성능이 가장 좋은 모델 로드
    report_to="none", # <-- 이 라인을 추가하여 wandb 연동 비활성화
)
print("TrainingArguments 설정 완료.")


# 학습에 필요한 클래스 및 함수들을 임포트합니다.
# Trainer: Hugging Face에서 제공하는 학습 유틸리티 클래스
# get_linear_schedule_with_warmup: 학습률 스케줄러
# EarlyStoppingCallback: 조기 종료 콜백
# AdamW: 옵티마이저 (가중치 감소가 적용된 Adam), torch.optim에서 임포트
# DataCollatorWithPadding: 데이터를 배치로 묶을 때 패딩을 적용하는 콜레이터 (이미 패딩했으므로 주석 처리됨)
from transformers import Trainer, get_linear_schedule_with_warmup, EarlyStoppingCallback
from torch.optim import AdamW # <-- AdamW를 torch.optim에서 임포트합니다.
# from transformers import DataCollatorWithPadding # 필요하다면 이 라인의 주석 해제

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # <-- 텍스트 패딩은 이미 완료되었습니다. 필요하다면 이 콜레이터를 사용하세요. (원문 주석)

# AdamW 옵티마이저를 모델의 파라미터와 학습률(2e-5)로 초기화합니다.
optimizer = AdamW(model.parameters(), lr=2e-5)

# 선형 학습률 스케줄러를 초기화합니다.
# optimizer: 사용할 옵티마이저
# num_warmup_steps: 웜업 스텝 수 (학습률이 선형적으로 증가하는 초기 단계)
# num_training_steps: 총 학습 스텝 수
# 총 학습 스텝 수는 (학습 데이터 샘플 수 / 배치 크기) * 에포크 수로 계산될 수 있지만, 여기서는 고정 값(12,500)으로 설정되었습니다.
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=2_500, num_training_steps=12_500)

# evaluate 라이브러리를 임포트합니다.
import evaluate # evaluate 라이브러리를 임포트합니다.

# 평가 지표로 "matthews_correlation" (매튜 상관계수)를 로드합니다.
# 참고: 다중 레이블 분류에는 F1-score (macro/micro), Precision, Recall 또는 각 레이블별 AUC-ROC 등이 더 적합할 수 있습니다.
# datasets.load_metric 대신 evaluate.load를 사용합니다.
print("평가 지표 로드 시작...")
metric = evaluate.load("matthews_correlation")
print("평가 지표 로드 완료.")

# 평가 지표를 계산하는 함수 정의
# eval_pred: 평가 결과 튜플 (로짓, 실제 레이블)
def compute_metrics(eval_pred):
    # 모델 출력 로짓과 실제 레이블을 가져옵니다.
    logits, labels = eval_pred
    # 로짓에서 가장 높은 값의 인덱스를 예측값으로 사용합니다. (다중 클래스 분류처럼 처리됨)
    # 다중 레이블 분류의 경우, 일반적으로 시그모이드 활성화 함수를 로짓에 적용한 후 임계값(threshold)을 기준으로 여러 개의 레이블을 예측합니다.
    # 현재 compute_metrics 함수는 다중 클래스 평가 지표 계산 방식에 가깝습니다.
    predictions = np.argmax(logits, axis=-1)
    # 예측값과 실제 레이블을 사용하여 매튜 상관계수를 계산하고 반환합니다.
    # evaluate 라이브러리의 compute 메서드는 딕셔너리를 반환할 수 있습니다.
    return metric.compute(predictions=predictions, references=labels)

print("Trainer 초기화 시작...")
# Trainer 객체를 초기화합니다.
trainer = Trainer(
    model=model, # 학습할 모델
    args=args, # 학습 인자
    train_dataset=tokenized_dataset["train"], # 학습 데이터셋
    eval_dataset=tokenized_dataset["validation"], # 평가 데이터셋
    tokenizer=tokenizer, # 토크나이저 (DataCollator 사용 시 필요)
    optimizers=(optimizer,scheduler), # 사용할 옵티마이저와 스케줄러 튜플
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.0)], # 조기 종료 콜백 설정
    compute_metrics=compute_metrics, # 평가 지표 계산 함수
#    data_collator=data_collator, # 데이터 콜레이터 (사용하지 않음)
)
print("Trainer 초기화 완료.")


# --- 학습 실행 ---

# 모델 학습을 시작합니다.
# 이 라인의 주석을 해제하고 코드를 실행하면 모델 학습이 시작됩니다.
print("모델 학습 시작...")
trainer.train()
print("모델 학습 종료.")

데이터셋 로드 시작...
데이터셋 로드 완료.
레이블 이진화 시작...
레이블 이진화 완료.
train_labels shape ::: (40000, 44)
test_labels shape :::: (5000, 44)
val_labels shape ::::: (5000, 44)

good!
바이너리 레이블 컬럼 추가 시작...
바이너리 레이블 컬럼 추가 완료.
split 컬럼 추가 시작...
split 컬럼 추가 완료.
'beomi/KcELECTRA-base' 토크나이저 로드 시작...
토크나이저 로드 완료.
map 함수 호출 전 데이터셋 컬럼 확인:
Train 스플릿 컬럼: ['ID', 'text', 'labels', 'binary_labels', 'split']
Validation 스플릿 컬럼: ['ID', 'text', 'labels', 'binary_labels', 'split']
Test 스플릿 컬럼: ['ID', 'text', 'labels', 'binary_labels', 'split']
map 함수를 사용한 데이터셋 전처리 시작...
map 함수를 사용한 데이터셋 전처리 완료.
데이터셋 형식 PyTorch 텐서로 변경 및 컬럼 정리 시작...
데이터셋 형식 변경 및 컬럼 정리 완료.
'beomi/KcELECTRA-base' 모델 로드 시작...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 로드 완료.
모델 설정에 레이블 정보 추가 완료.
TrainingArguments 설정 시작...
TrainingArguments 설정 완료.
평가 지표 로드 시작...
평가 지표 로드 완료.
Trainer 초기화 시작...
Trainer 초기화 완료.
모델 학습 시작...


<ipython-input-17-db803eaf2f18>:385: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
